In [64]:
# Cryptanalysis - Task 4 - MITM - v1

# TO-DO:
#  - implement iterative attack until the key guess is unique (+ stats?)
#  - answer question from the task paper

# Remarks:

size = 2**10
P = Permutations(size)
p = P.random_element()  # very probably non-linear)
#print(p)
#print(p[0], p[15])
#print(list(p))
#p = p -1
#print(list(p) - 1)


def enc(mes, key):
    result = (p[mes ^^ key] - 1) ^^ key
    #if result >= 16:
    #    print(mes, key)
    return result
    
def dec(ct, key):
    result = p.index((ct ^^ key) + 1) ^^ key
    return result

def EDE_2(mes, keys):
    result = enc(dec(enc(mes, keys[0]), keys[1]), keys[0])
    return result

def DED_2(mes, keys):
    result = dec(enc(dec(mes, keys[0]), keys[1]), keys[0])
    return result

def attack(keys, a):
    k0_guesses = range(size)
    #k0_guesses = range(2**12)
    #k0_guesses = range(2**5)
    dict_0 = {}

    for k0_guess in k0_guesses:  # k0_guess ~ i
        #a = 0
        m_i = dec(a, k0_guess)
        
        if m_i not in dict_0.keys():
            #print("match")
            dict_0[m_i] = [[k0_guess], []]
        else:
            dict_0[m_i][0].append(k0_guess)
        
        c_i = EDE_2(m_i, keys)
        b_i = dec(c_i, k0_guess)
        
        if b_i not in dict_0.keys():
            dict_0[b_i] = [[], [k0_guess]]
        else:
            dict_0[b_i][1].append(k0_guess)
            #print("match:", k0_guess, dict_0[b_i][0])
    
    final_guesses = []
    for key in dict_0.keys():
        vals = dict_0[key]
        if len(vals[0]) != 0 and len(vals[1]) != 0:
            for i in range(len(vals[0])):
                for j in range(len(vals[1])):
                    final_guesses.append((vals[0][i], vals[1][j]))
    return final_guesses, dict_0

def test_attack(iterations):
    found = 0
    total_guesses = 0
    for i in range(iterations):
        k0, k1 = randint(0, size - 1), randint(0, size - 1)
        res, _ = attack([k0, k1])
        if (k1, k0) in res:
            found += 1
            total_guesses += len(res)
    if found != 0:
        avg_guesses = float(total_guesses / found)
    else:
        avg_guesses = 0
    return found, avg_guesses

mes_1 = 13
#key_0, key_1 = 12, 7
key_0, key_1 = randint(0, size - 1), randint(0, size - 1)
print("keys:", key_0, key_1)
keys_1 = [key_0, key_1]
enc_1 = enc(mes_1, keys_1[0])
dec_1 = dec(enc_1, keys_1[0])

ede_1 = EDE_2(mes_1, keys_1)
ded_1 = DED_2(ede_1, keys_1)
#print("enc + dec:", mes_1, "->", enc_1, "->", dec_1)
#print("EDE + DED:", mes_1, "->", ede_1, "->", ded_1)

#result, table = attack(keys_1, 0)
#print(result)
#print("Correct inside:", (key_1, key_0) in result)
#print("Num of guesses:", len(result), "/", size*size)

res_0, _ = attack(keys_1, 0)
res_1, _ = attack(keys_1, 1)

print(set(res_0).intersection(res_1))

#iterations = 10
#found, avg = test_attack(iterations)
#print("found:", found, "/", iterations, "avg:", avg)

keys: 401 257
{(68, 81), (257, 401), (32, 692)}
